In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
import numpy as np
max_long = 25

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
import sys
sys.path.append('/content/gdrive/My Drive')

In [7]:
!pip install unidecode

     |████████████████████████████████| 235 kB 4.9 MB/s 


In [8]:
import data
data.get_data, data.clean_data

(<function data.get_data>, <function data.clean_data>)

In [9]:
import token_glove
token_glove.create_list, token_glove.token_ize, token_glove.voc_token, token_glove.dict_token, token_glove.token_tweet, token_glove.glove_vector

(<function token_glove.create_list>,
 <function token_glove.token_ize>,
 <function token_glove.voc_token>,
 <function token_glove.dict_token>,
 <function token_glove.token_tweet>,
 <function token_glove.glove_vector>)

In [10]:
import metrics
metrics.recall_m, metrics.precision_m, metrics.f1_m

(<function metrics.recall_m>,
 <function metrics.precision_m>,
 <function metrics.f1_m>)

In [11]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [12]:
df = data.get_data(addon=True)
cleaned_df = data.clean_data(df)

In [12]:
import joblib
text = token_glove.create_list(cleaned_df['text'])
tokenizer = joblib.load('/content/gdrive/My Drive/token_cam_test.joblib')
#token = token_glove.token_ize(text)
X = token_glove.token_tweet(text, tokenizer)

In [13]:
import joblib
text = token_glove.create_list(cleaned_df['text'])
token = token_glove.token_ize(text)
X = token_glove.token_tweet(text, token)

In [54]:
text = cleaned_df.iloc[3:5,1]

In [55]:
text

3    people receive wildfire evacuation order calif...
4          got sent photo alaska smoke wildfire school
Name: text, dtype: object

In [56]:
text = text.tolist()

In [57]:
text

['people receive wildfire evacuation order california',
 'got sent photo alaska smoke wildfire school']

In [37]:
import joblib
tokenizer = joblib.load('/content/gdrive/My Drive/token_cam_test.joblib')

In [ ]:
tokenizer = joblib.load('/content/gdrive/My Drive/token_cam_test.joblib')

In [58]:
text = tokenizer.texts_to_sequences(text)

In [59]:
text = pad_sequences(text, maxlen=25, padding='post')

In [60]:
text

array([[   5, 2511,  172,  373,  435,  154,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0],
       [  35,  857,  289, 2878,  173,  172,  109,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0]], dtype=int32)

In [14]:
word_vector_matrix = token_glove.glove_vector(token)

In [15]:
word_vector_matrix

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [-0.015537,  0.11158 , -0.23599 , ..., -0.037577, -0.13539 ,
         0.45965 ],
       [-0.12293 ,  0.20598 ,  0.056996, ...,  0.45836 ,  0.13722 ,
         0.66997 ],
       ...,
       [ 0.88527 , -0.22375 ,  0.35806 , ...,  0.41284 ,  0.71352 ,
        -0.67698 ],
       [ 0.22705 , -0.63495 , -0.082351, ...,  0.28127 , -0.22173 ,
        -0.20049 ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ]])

In [44]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Activation, Dropout, LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model, save_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
vocab_size = token_glove.voc_token(token)
vec_size = 200

In [16]:
y = cleaned_df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size= 0.3, stratify = y)

In [22]:
X_train

array([[1321,  784,  261, ...,    0,    0,    0],
       [4478,   25,  937, ...,    0,    0,    0],
       [ 156, 3418,  102, ...,    0,    0,    0],
       ...,
       [ 646,   24, 1937, ...,    0,    0,    0],
       [1984, 6348, 3695, ...,    0,    0,    0],
       [ 138, 3458,  371, ...,    0,    0,    0]], dtype=int32)

In [23]:
y_train

907      0
10281    0
7235     0
4797     0
7060     0
        ..
3006     1
2534     0
1396     0
8739     0
10784    0
Name: target, Length: 13269, dtype: int64

In [42]:
model4 = Sequential()

es = EarlyStopping(patience=5,restore_best_weights= True)

model4.add(Embedding(vocab_size, vec_size, input_length=max_long, weights = [word_vector_matrix], trainable = True))
model4.add(LSTM(256,return_sequences = True))
model4.add(Dropout(0.5))
model4.add(LSTM(128,return_sequences = True))
model4.add(Dropout(0.5))
model4.add(LSTM(64, return_sequences = True))
model4.add(Dropout(0.5))
model4.add(LSTM(32, return_sequences = False))
model4.add(Dropout(0.4))
model4.add(Dense(256, activation = 'relu'))
model4.add(Dropout(0.2))
model4.add(Dense(16, activation = 'relu'))
model4.add(Dropout(0.1))  
model4.add(Dense(1, activation = 'sigmoid'))
model4.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate = 0.0001),metrics=[metrics.f1_m])
history2 = model4.fit(X_train, y_train, batch_size=16, epochs = 2, validation_data = (X_test, y_test), callbacks = [es])

NameError: ignored

In [41]:
model4.predict(text)

NameError: ignored

In [21]:
import joblib
model4.save('/content/gdrive/My Drive/RNN_keras_camille1')
#joblib.dump(model4, '/content/gdrive/My Drive/modelRNN.joblib')

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/RNN_keras_camille1/assets


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/RNN_keras_camille1/assets


In [46]:
model_test = load_model('/content/gdrive/My Drive/RNN_keras_camille1', custom_objects={'f1_m': metrics.f1_m})

In [61]:
model_test.predict(text)

array([[0.80679613],
       [0.81136656]], dtype=float32)